# Lecture 4 Word Window Classification and Neural Networks

这节课介绍了根据上下文预测单词分类的问题，与常见神经网络课程套路不同，以间隔最大化为目标函数，推导了对权值矩阵和词向量的梯度；初步展示了与传统机器学习方法不一样的风格。

## 分类问题

给定训练集
$$\{x^{(i)},y^{(i)}\}_{1}^{N}$$

其中$x^{(i)}$是一个$d-$维向量，$y^{(i)}$是一个$C-$维one-hot向量，$N$是总数。在传统的机器学习方法中，往往通过诸如逻辑斯谛回归和SVM找到分类决策边界：

![屏幕快照 2018-05-08 下午6.26.01.png](https://i.loli.net/2018/05/08/5af17b49f2e25.png)

### softmax细节

softmax分类函数：

![屏幕快照 2018-05-08 下午6.27.41.png](https://i.loli.net/2018/05/08/5af17bab765b8.png)

### softmax与交叉熵误差

训练的时候，可以直接最小化正确类别的概率的负对数：

![屏幕快照 2018-05-08 下午7.49.05.png](https://i.loli.net/2018/05/08/5af18ee6d101c.png)

其实这个损失函数等效于交叉熵：

![屏幕快照 2018-05-08 下午7.49.38.png](https://i.loli.net/2018/05/08/5af18ee6db81a.png)

这是因为类别是one-hot向量。

对N个数据点来讲有：

![屏幕快照 2018-05-08 下午7.49.30.png](https://i.loli.net/2018/05/08/5af18ee6e251b.png)

加上正则化项有：

![屏幕快照 2018-05-08 下午7.49.25.png](https://i.loli.net/2018/05/08/5af18ee6e835f.png)

![屏幕快照 2018-05-08 下午7.50.58.png](https://i.loli.net/2018/05/08/5af18f2fb2ecb.png)

红线是test error，蓝线是training error，横轴是模型复杂度或迭代次数。直线是方差偏差均衡

### 优化

一般的ML问题中，参数由权值矩阵的列组成维度不会太大。而在词向量或其他深度学习中，需要同时学习权值矩阵和词向量。参数一多，就容易过拟合：

![屏幕快照 2018-05-08 下午7.53.07.png](https://i.loli.net/2018/05/08/5af18fb2c9cc9.png)

### re-training词向量失去泛化效果

比如有一个给单词做情感分析的小任务，在预训练的词向量中，这三个表示电视的单词都是在一起的：

![屏幕快照 2018-05-08 下午7.54.00.png](https://i.loli.net/2018/05/08/5af1901269416.png)

但由于情感分析语料中，训练集只含有TV和telly，导致re-training之后两者跑到别处去了：

![屏幕快照 2018-05-08 下午7.54.17.png](https://i.loli.net/2018/05/08/5af190126b759.png)

于是在测试集上导致television被误分类。

这个例子说明，如果任务的语料非常小，则不必在任务语料上重新训练词向量，否则会导致词向量过拟合。

顺便介绍一下词向量相关的术语：

词向量矩阵L常被称作lookup table：

![屏幕快照 2018-05-08 下午7.54.35.png](https://i.loli.net/2018/05/08/5af1901272d48.png)

Word vectors = word embeddings = word representations (mostly)

可能有些人认为“词向量”是个很俗的翻译，但根据这门课的课件，其实与“词嵌入”是等价的。而且Rocher也几乎是一直在用word vector，而不是其他术语。


## 窗口分类
这是一种根据上下文给单个单词分类的任务，可以用于消歧或命名实体分类。上下文Window的向量可以通过拼接所有窗口中的词向量得到：

![屏幕快照 2018-05-09 下午3.14.32.png](https://i.loli.net/2018/05/09/5af29fedb979f.png)

这是一个

![屏幕快照 2018-05-09 下午3.14.24.png](https://i.loli.net/2018/05/09/5af29fede8c33.png)

的列向量。

### 最简单的分类器：softmax

![屏幕快照 2018-05-28 下午4.54.19.png](https://i.loli.net/2018/05/28/5b0bc3f46ac9e.png)

$J$对$x$求导，注意这里的$x$指的是窗口所有单词的词向量拼接向量。

![屏幕快照 2018-05-28 下午4.54.33.png](https://i.loli.net/2018/05/28/5b0bc3f3e7368.png)

于是就可以更新词向量了：

![屏幕快照 2018-05-28 下午4.54.43.png](https://i.loli.net/2018/05/28/5b0bc3f3be8f2.png)

另一方面，对$W$求偏导数，将所有参数的偏导数写到一起有：

![屏幕快照 2018-05-28 下午4.54.48.png](https://i.loli.net/2018/05/28/5b0bc3f3d008e.png)

在更新过程中计算代价最高的有矩阵运算$f=W_{x}$和指数函数，但矩阵运算比循环要快多了（在CPU上快一个数量级）。

### softmax（等价于逻辑斯谛回归）效果有限

仅限于较小的数据集，能够提供一个勉强的线性分类决策边界。

![屏幕快照 2018-05-28 下午6.31.41.png](https://i.loli.net/2018/05/28/5b0bdb5d07b55.png)


## 使用神经网络

神经网络可以提供非线性的决策边界：

![屏幕快照 2018-05-28 下午6.33.09.png](https://i.loli.net/2018/05/28/5b0bdb5c9615a.png)

### 从逻辑斯谛回归到神经网络

老生常谈了，一些术语：

![屏幕快照 2018-05-28 下午6.33.48.png](https://i.loli.net/2018/05/28/5b0bdb5d0c8e1.png)

每个神经元是一个二分类逻辑斯谛回归单元：

![屏幕快照 2018-05-28 下午6.34.02.png](https://i.loli.net/2018/05/28/5b0bdb5d0ae70.png)

神经网络同时运行多个逻辑斯谛回归，但不需要提前指定它们具体预测什么：

![屏幕快照 2018-05-28 下午6.34.09.png](https://i.loli.net/2018/05/28/5b0bded11b469.png)

我们把预测结果喂给下一级逻辑斯谛回归单元，由损失函数自动决定它们预测什么：

![屏幕快照 2018-05-28 下午6.34.17.png](https://i.loli.net/2018/05/28/5b0bded133bda.png)

于是就得到了一个多层网络：

![屏幕快照 2018-05-28 下午6.34.27.png](https://i.loli.net/2018/05/28/5b0bded13b1b1.png)

### 为什么需要非线性

因为线性系统所有层等效于一层：$$W_{1}W_{2}x=Wx$$而非线性模型可以捕捉很复杂的数据：

![屏幕快照 2018-05-29 下午1.44.18.png](https://i.loli.net/2018/05/29/5b0ce99e96a23.png)

### 前向传播网络

一个简单的网络：

![屏幕快照 2018-05-29 下午1.45.15.png](https://i.loli.net/2018/05/29/5b0ce99e9cbf2.png)

这种红点图经常在论文里看到，大致代表单元数；中间的空格分隔开一组神经元，比如隐藏层单元数为2×4。$U$是隐藏层到class的权值矩阵：

![屏幕快照 2018-05-29 下午1.46.06.png](https://i.loli.net/2018/05/29/5b0ce99ea8a63.png)

其中$a$是激活函数：

![屏幕快照 2018-05-29 下午1.46.44.png](https://i.loli.net/2018/05/29/5b0ce99e59b36.png)

### 间隔最大化目标函数

怎么设计目标函数呢，记$s_{c}$代表误分类样本的得分，$s$表示正确分类样本的得分。则朴素的思路是最大化$(s−s_{c}) $或最小化$(s_{c}−s)$。但有种方法只计算$s_{c}>s\Rightarrow(s_{c}−s)>0$时的错误，也就是说我们只要求正确分类的得分高于错误分类的得分即可，并不要求错误分类的得分多么多么小。这得到间隔最大化目标函数：$$minimizeJ=max(s_{c}−s,0)$$但上述目标函数要求太低，风险太大了，没有留出足够的“缓冲区域”。可以指定该间隔的宽度$(s−s_{c}<\Delta)$ ，得到：$$minimizeJ=max(\Delta+s_{c}−s,0)$$可以调整其他参数使得该间隔为1：$$minimizeJ=max(1+s_{c}−s,0)$$

这实际上是将函数间隔转换为几何间隔，参考SVM：http://www.hankcs.com/ml/support-vector-machine.html#h3-3

在这个分类问题中，这两个得分的计算方式为：$s_{c}=U^{T}f(W_{x}+b)$ 和 $s=U^{T}f(W_{x}+b)$；通常通过负采样算法得到负例。

另外，这个目标函数的好处是，随着训练的进行，可以忽略越来越多的实例，而只专注于那些难分类的实例。



## 反向传播训练

依然是老生常谈，跳过链式法则推导直接贴结果：

![屏幕快照 2018-05-29 下午2.14.52.png](https://i.loli.net/2018/05/29/5b0cf37b138a3.png)

其中$\delta^{(k)}$是第k层的误差：$$\delta(k)=f^{\prime}(z^{(k)})\circ(W^{(k)T}\delta^{(k+1)})$$
可见只要控制误差的计算方式，就可以smoothly地过渡到间隔最大化目标函数：

![屏幕快照 2018-05-29 下午2.29.00.png](https://i.loli.net/2018/05/29/5b0cf37b24c43.png)

另外，对偏置的偏导数是$\delta^{(k)} _{i}$：

![屏幕快照 2018-05-29 下午2.29.56.png](https://i.loli.net/2018/05/29/5b0cf37b1f658.png)

最后一片拼图是对词向量的偏导数，由于连接时每个输入单元连到了多个隐藏单元，所以对某个输入单元的偏导数是求和的形式（残差来自相连的隐藏单元）：

![屏幕快照 2018-05-29 下午2.31.55.png](https://i.loli.net/2018/05/29/5b0cf441c9ff3.png)

其中，$W^{T} _{⋅j}$是第$j$列，转置后得到行向量；红色部分是误差，相乘后得到一个标量，代表词向量第j维的导数。那么对整个词向量的偏导数就是：

![屏幕快照 2018-05-29 下午2.33.20.png](https://i.loli.net/2018/05/29/5b0cf44194473.png)